In [ ]:
pip install --upgrade scikit-learn

In [ ]:
pip install --upgrade imbalanced-learn

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split

# ================================
# Load and preprocess dataset
# ================================
def load_dataset(directory, target_size=(224, 224)):
    datagen = ImageDataGenerator(rescale=1./255)
    generator = datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=2000,  # Load large batch
        class_mode='categorical',
        shuffle=True
    )

    images, labels = next(generator)
    class_mapping = generator.class_indices
    inv_class_mapping = {v: k for k, v in class_mapping.items()}
    return images, labels, inv_class_mapping

# Load dataset
images, labels, inv_class_mapping = load_dataset("/kaggle/input/corn-or-maize-leaf-disease-dataset/data")

# Convert to class indices for SMOTE
class_indices = np.argmax(labels, axis=1)

# Train-test split
X_train, X_test, y_train_indices, y_test_indices = train_test_split(
    images, class_indices, test_size=0.05, stratify=class_indices, random_state=42
)

# Flatten images for SMOTE
n_samples, height, width, channels = X_train.shape
X_train_flat = X_train.reshape(n_samples, height * width * channels)

# Apply SMOTE
print("Before SMOTE - Class distribution:", np.bincount(y_train_indices))
smote = SMOTE(random_state=42)
X_train_balanced_flat, y_train_balanced = smote.fit_resample(X_train_flat, y_train_indices)
X_train_balanced = X_train_balanced_flat.reshape(-1, height, width, channels)

# One-hot encode labels
y_train_balanced = keras.utils.to_categorical(y_train_balanced, num_classes=len(inv_class_mapping))
y_test = keras.utils.to_categorical(y_test_indices, num_classes=len(inv_class_mapping))

print("After SMOTE - Class distribution:", np.bincount(np.argmax(y_train_balanced, axis=1)))

# ================================
# Build Model
# ================================
def create_model():
    base_model = keras.applications.MobileNetV2(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    inputs = keras.Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(len(inv_class_mapping), activation='softmax')(x)

    return keras.Model(inputs, outputs)

model = create_model()

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ================================
# Train model
# ================================
history = model.fit(
    X_train_balanced, y_train_balanced,
    epochs=50,
    validation_data=(X_test, y_test),
    batch_size=16
)



# ================================
# Save the model
# ================================
model.save('maize_mobilenet.keras')

2025-07-20 04:47:21.249616: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752986841.639886      86 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752986841.748102      86 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 4188 images belonging to 4 classes.
Before SMOTE - Class distribution: [516 596 269 519]
After SMOTE - Class distribution: [596 596 596 596]


I0000 00:00:1752986888.757905      86 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1752986888.758687      86 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/50


I0000 00:00:1752986901.868045     151 service.cc:148] XLA service 0x7ea28c095330 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752986901.869181     151 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1752986901.869199     151 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1752986902.752639     151 cuda_dnn.cc:529] Loaded cuDNN version 90300


  6/149 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.4099 - loss: 1.6029 

I0000 00:00:1752986907.682058     151 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


149/149 ━━━━━━━━━━━━━━━━━━━━ 22s 61ms/step - accuracy: 0.7196 - loss: 0.6964 - val_accuracy: 0.8700 - val_loss: 0.2948
Epoch 2/50
149/149 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9047 - loss: 0.2597 - val_accuracy: 0.9000 - val_loss: 0.2855
Epoch 3/50
149/149 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9171 - loss: 0.2048 - val_accuracy: 0.8800 - val_loss: 0.3199
Epoch 4/50
149/149 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9425 - loss: 0.1722 - val_accuracy: 0.8800 - val_loss: 0.2934
Epoch 5/50
149/149 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9380 - loss: 0.1569 - val_accuracy: 0.9000 - val_loss: 0.3160
Epoch 6/50
149/149 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.9533 - loss: 0.1281 - val_accuracy: 0.8900 - val_loss: 0.3170
Epoch 7/50
149/149 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9488 - loss: 0.1276 - val_accuracy: 0.8900 - val_loss: 0.3784
Epoch 8/50
149/149 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9674 - loss: 0.0916 - val_accuracy: 0.90

In [2]:
inv_class_mapping

{0: 'Blight', 1: 'Common_Rust', 2: 'Gray_Leaf_Spot', 3: 'Healthy'}